In [1]:
import pandas as pd
import h5py
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn
seaborn.set()

In [4]:
keys = ["hydro_07ED002", "hydro_08EB003", "hydro_08EC001", 'hydro_08GA030', "hydro_08GA077",
        "hydro_08GA079", "hydro_08GB013", "hydro_08HA009", "hydro_08HB082", "hydro_08HC006",
        "hydro_08HD025", "hydro_08JA023", "hydro_08LC041", "hydro_08MH141", "hydro_08MH148",
        "hydro_08MH149", "hydro_08MH167", "hydro_08NA002", "hydro_08NB017", "hydro_08ND017",
        "hydro_08NE102", "hydro_08NE103", "hydro_08NE104", "hydro_08NE126", "hydro_08NH064",
        "hydro_08NH127", "hydro_08NM202", "hydro_08NM212", "hydro_08NM213","hydro_08NM234"]

In [5]:
for key in keys:
        daily_hdf = pd.read_hdf('daily_combined_data.hdf', key=key)
        print(key, len(daily_hdf), len(daily_hdf.dropna()))

hydro_07ED002 5555 5555
hydro_08EB003 3811 3811
hydro_08EC001 10665 10665
hydro_08GA030 2667 2667
hydro_08GA077 2896 2896
hydro_08GA079 2922 2922
hydro_08GB013 2541 2541
hydro_08HA009 23017 23017
hydro_08HB082 9361 9361
hydro_08HC006 3247 0
hydro_08HD025 7567 7567
hydro_08JA023 1095 1095
hydro_08LC041 16655 16655
hydro_08MH141 2918 2918
hydro_08MH148 4611 4611
hydro_08MH149 12475 12475
hydro_08MH167 3624 3624
hydro_08NA002 11894 11894
hydro_08NB017 13558 13558
hydro_08ND017 16308 16308
hydro_08NE102 17810 17810
hydro_08NE103 7305 7305
hydro_08NE104 17752 17752
hydro_08NE126 0 0
hydro_08NH064 31655 31655
hydro_08NH127 18873 18873
hydro_08NM202 0 0
hydro_08NM212 0 0
hydro_08NM213 1601 1601
hydro_08NM234 196 196


In [6]:
for key in keys:
        monthly_hdf = pd.read_hdf('monthly_combined_data.hdf', key=key)
        print(key, len(monthly_hdf), len(monthly_hdf.dropna()))

hydro_07ED002 182 0
hydro_08EB003 13 0
hydro_08EC001 188 0
hydro_08GA030 0 0
hydro_08GA077 0 0
hydro_08GA079 0 0
hydro_08GB013 0 0
hydro_08HA009 470 0
hydro_08HB082 161 0
hydro_08HC006 12 0
hydro_08HD025 114 0
hydro_08JA023 36 0
hydro_08LC041 441 0
hydro_08MH141 0 0
hydro_08MH148 149 0
hydro_08MH149 263 0
hydro_08MH167 0 0
hydro_08NA002 238 0
hydro_08NB017 312 0
hydro_08ND017 390 0
hydro_08NE102 449 0
hydro_08NE103 240 0
hydro_08NE104 449 0
hydro_08NE126 0 0
hydro_08NH064 896 0
hydro_08NH127 478 0
hydro_08NM202 0 0
hydro_08NM212 0 0
hydro_08NM213 15 0
hydro_08NM234 0 0
